## Data Exploration
Preliminary notebook to get a "feel" of the data before I create the ETL script

In [ ]:
# notes from instructions:

In [194]:
import pandas as pd

In [195]:
# constituent information
pd.read_csv(r'data/cons.csv').columns

Index(['cons_id', 'prefix', 'firstname', 'middlename', 'lastname', 'suffix',
       'salutation', 'gender', 'birth_dt', 'title', 'employer', 'occupation',
       'income', 'source', 'subsource', 'userid', 'password', 'is_validated',
       'is_banned', 'change_password_next_login', 'consent_type_id',
       'create_dt', 'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note'],
      dtype='object')

In [196]:
df_cons=pd.read_csv(r'data/cons.csv', usecols=['cons_id', 'firstname',
                                               'middlename', 'lastname',
                                              'source', 'subsource', 'create_dt', 'modified_dt'])
print(df_cons.columns)
df_cons.head(3)

Index(['cons_id', 'firstname', 'middlename', 'lastname', 'source', 'subsource',
       'create_dt', 'modified_dt'],
      dtype='object')


,cons_id,firstname,middlename,lastname,source,subsource,create_dt,modified_dt
0,1,NaN,Lee,NaN,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
1,2,NaN,NaN,NaN,facebook,pRzBAZSGNScwCyreCEYr,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57"
2,3,NaN,David,King,NaN,UAWXnALxxBXmwbPibFdw,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57"


In [197]:
df_cons[['source', 'subsource']][:5]

,source,subsource
0,google,NaN
1,facebook,pRzBAZSGNScwCyreCEYr
2,NaN,UAWXnALxxBXmwbPibFdw
3,google,NaN
4,NaN,NaN


In [198]:
# constituent email addresses
# note: bool cols (including is_primary) are all 1/0 for True/False

In [199]:
emails=pd.read_csv(r'data/cons_email.csv', usecols=['cons_email_id', 'cons_id',
                                                    'cons_email_type_id',
                                                    'is_primary', 'email',])
print(emails.columns)
emails.head(3)

Index(['cons_email_id', 'cons_id', 'cons_email_type_id', 'is_primary',
       'email'],
      dtype='object')


,cons_email_id,cons_id,cons_email_type_id,is_primary,email
0,1,548198,3361,1,xmartinez@vincent.com
1,2,491137,2474,1,hmiller@haynes.biz
2,3,413429,5175,1,aaron64@yahoo.com


In [200]:
# constituent subscription status
# if email NULL or not present, assumed to be still sub'd
# only care about subscription status where chapter_id is 1

In [93]:
subs = pd.read_csv(r'data/cons_email_chapter_subscription.csv')
print(subs.columns)
subs.head(3)

Index(['cons_email_chapter_subscription_id', 'cons_email_id', 'chapter_id',
       'isunsub', 'unsub_dt', 'modified_dt'],
      dtype='object')


,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub,unsub_dt,modified_dt
0,1,332188,1,1,"Sat, 1971-06-12 15:38:44","Thu, 1990-06-28 10:54:20"
1,2,536526,1,1,"Wed, 2006-07-12 01:50:45","Thu, 1979-09-20 06:02:35"
2,3,134711,1,1,"Tue, 1987-01-06 13:05:15","Sun, 1974-03-03 15:11:50"


In [115]:
subs = subs[subs.chapter_id==1]
(subs.chapter_id==1).all()

True

In [ ]:
# join subs and email list on cons_email_id

In [127]:
subs_emails = subs.join(emails.set_index('cons_email_id'),
                on='cons_email_id',
                how='left',
                lsuffix='_sub_status', rsuffix='_email')


In [209]:
# there are no blank emails
(subs_emails['email'].str.isspace()).any()

False

In [210]:
# there are no null emails
(subs_emails['email'].isnull()).any()

False

In [201]:
# LEFT join == LEFT OUTER
print(len(subs_emails))
print(len(subs))

275484
275484


In [215]:
# check for accuracy
subs_emails

,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub,unsub_dt,modified_dt,cons_id,cons_email_type_id,is_primary,email
0,1,332188,1,1,"Sat, 1971-06-12 15:38:44","Thu, 1990-06-28 10:54:20",558514,2582,1,santiagoeric@hunt.info
1,2,536526,1,1,"Wed, 2006-07-12 01:50:45","Thu, 1979-09-20 06:02:35",12447,9716,0,amandabullock@cunningham-brown.com
2,3,134711,1,1,"Tue, 1987-01-06 13:05:15","Sun, 1974-03-03 15:11:50",49952,1757,1,michael06@torres-baker.com
3,4,660345,1,1,"Sat, 2016-08-06 11:06:09","Wed, 1995-09-13 23:45:03",27409,830,0,rosalessteven@yahoo.com
4,5,184268,1,1,"Sun, 2000-05-28 02:20:45","Sat, 1983-12-10 08:09:58",76099,9744,1,matthewjohnson@vazquez.com
5,6,621735,1,1,"Wed, 2009-12-09 14:08:54","Sat, 1990-09-01 06:53:49",634103,7193,0,qmcdaniel@hotmail.com
6,7,449159,1,1,"Fri, 2001-02-16 03:25:34","Fri, 2018-04-27 17:26:03",386356,6244,0,castillosherri@yahoo.com
7,8,684697,1,1,"Wed, 1988-06-29 09:32:53","Mon, 1978-06-12 17:50:09",190419,705,1,thomassanchez@harris-park.com
8,9,247669,1,1,"Tue, 2003-07-08 23:52:27","Wed, 1972-08-09 06:43:57",675032,7171,0,igonzales@odom.biz
9,10,298884,1,1,"Sat, 1986-09-20 16:39:10","Wed, 1974-09-25 07:52:08",622465,7159,1,cynthiaprice@hotmail.com


In [216]:
emails[emails.cons_email_id==216591]

,cons_email_id,cons_id,cons_email_type_id,is_primary,email
216590,216591,538622,2305,0,lyonslucas@gmail.com


In [211]:
# join subs_emails with cons to get person creation / update time
alldata = subs_emails.join(df_cons.set_index('cons_id'), on='cons_id'
                          , how='left', lsuffix='subsemails',rsuffix='cons')
alldata

,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub,unsub_dt,modified_dtsubsemails,cons_id,cons_email_type_id,is_primary,email,firstname,middlename,lastname,source,subsource,create_dt,modified_dtcons
0,1,332188,1,1,"Sat, 1971-06-12 15:38:44","Thu, 1990-06-28 10:54:20",558514,2582,1,santiagoeric@hunt.info,NaN,Molly,Cobb,NaN,CFoxSKPolgsqBnRMvDMk,"Sun, 2002-03-17 02:35:08","Sat, 2011-03-26 20:08:01"
1,2,536526,1,1,"Wed, 2006-07-12 01:50:45","Thu, 1979-09-20 06:02:35",12447,9716,0,amandabullock@cunningham-brown.com,NaN,Amy,Adkins,organic,zcqRLbWEZMTlwOaWmbug,"Sat, 2012-12-15 00:23:24","Wed, 2010-03-03 12:38:21"
2,3,134711,1,1,"Tue, 1987-01-06 13:05:15","Sun, 1974-03-03 15:11:50",49952,1757,1,michael06@torres-baker.com,Victor,NaN,Morrison,google,uPKiNNVkbftdwIeuvHkl,"Thu, 1987-12-17 00:22:11","Mon, 2012-11-26 18:07:20"
3,4,660345,1,1,"Sat, 2016-08-06 11:06:09","Wed, 1995-09-13 23:45:03",27409,830,0,rosalessteven@yahoo.com,Jennifer,Zachary,Scott,twitter,BGLlcgUlxhlKUIeQAkjN,"Sun, 1989-07-16 22:14:29","Sun, 1980-08-10 09:19:53"
4,5,184268,1,1,"Sun, 2000-05-28 02:20:45","Sat, 1983-12-10 08:09:58",76099,9744,1,matthewjohnson@vazquez.com,NaN,Gregory,Fleming,facebook,hnRivrBsTZsuYGDchodE,"Tue, 1976-07-27 11:26:08","Mon, 2004-05-03 18:40:04"
5,6,621735,1,1,"Wed, 2009-12-09 14:08:54","Sat, 1990-09-01 06:53:49",634103,7193,0,qmcdaniel@hotmail.com,NaN,NaN,NaN,organic,NaN,"Thu, 2004-12-02 19:05:22","Tue, 2000-06-27 07:18:47"
6,7,449159,1,1,"Fri, 2001-02-16 03:25:34","Fri, 2018-04-27 17:26:03",386356,6244,0,castillosherri@yahoo.com,NaN,Julie,NaN,NaN,XlBNUqpdOXNdABPdcmEk,"Sun, 1972-05-28 06:31:49","Wed, 2006-02-22 12:52:35"
7,8,684697,1,1,"Wed, 1988-06-29 09:32:53","Mon, 1978-06-12 17:50:09",190419,705,1,thomassanchez@harris-park.com,NaN,Peter,NaN,twitter,myNeTwGCcoeKiiSgbOLa,"Sat, 1997-08-16 03:42:55","Fri, 1971-03-05 21:36:16"
8,9,247669,1,1,"Tue, 2003-07-08 23:52:27","Wed, 1972-08-09 06:43:57",675032,7171,0,igonzales@odom.biz,NaN,Ronald,NaN,facebook,MzwVhifKEKwtnoHqozJP,"Sun, 1986-01-12 08:58:25","Sun, 1982-01-03 14:45:30"
9,10,298884,1,1,"Sat, 1986-09-20 16:39:10","Wed, 1974-09-25 07:52:08",622465,7159,1,cynthiaprice@hotmail.com,Kevin,Beth,Pearson,twitter,NaN,"Sun, 2014-05-11 13:36:02","Wed, 2017-11-08 05:43:32"


In [213]:
(alldata['chapter_id']==1).all()

True

In [149]:
# convert 1/0 columns to booleans
alldata.isunsub = alldata.isunsub.apply(lambda x: x==True)

In [156]:
alldata=alldata.rename(columns={"source": "code", "isunsub": "is_unsub",
                                "create_dt": "created_dt", "modified_dtcons": "updated_dt"})

In [157]:
alldata.to_csv('people.csv',index=False, columns=['email','code','is_unsub','created_dt','updated_dt'])

In [184]:
alldata=pd.read_csv('people.csv')

In [185]:
alldata

,email,code,is_unsub,created_dt,updated_dt
0,santiagoeric@hunt.info,NaN,True,"Sun, 2002-03-17 02:35:08","Sat, 2011-03-26 20:08:01"
1,amandabullock@cunningham-brown.com,organic,True,"Sat, 2012-12-15 00:23:24","Wed, 2010-03-03 12:38:21"
2,michael06@torres-baker.com,google,True,"Thu, 1987-12-17 00:22:11","Mon, 2012-11-26 18:07:20"
3,rosalessteven@yahoo.com,twitter,True,"Sun, 1989-07-16 22:14:29","Sun, 1980-08-10 09:19:53"
4,matthewjohnson@vazquez.com,facebook,True,"Tue, 1976-07-27 11:26:08","Mon, 2004-05-03 18:40:04"
5,qmcdaniel@hotmail.com,organic,True,"Thu, 2004-12-02 19:05:22","Tue, 2000-06-27 07:18:47"
6,castillosherri@yahoo.com,NaN,True,"Sun, 1972-05-28 06:31:49","Wed, 2006-02-22 12:52:35"
7,thomassanchez@harris-park.com,twitter,True,"Sat, 1997-08-16 03:42:55","Fri, 1971-03-05 21:36:16"
8,igonzales@odom.biz,facebook,True,"Sun, 1986-01-12 08:58:25","Sun, 1982-01-03 14:45:30"
9,cynthiaprice@hotmail.com,twitter,True,"Sun, 2014-05-11 13:36:02","Wed, 2017-11-08 05:43:32"


### make the groupby create-date and sum dataframe
(assumption is that we're still working the dataset where "we only care about chapter_id = 1" data)

In [189]:
alldata['acquisitions']=1
alldata['acquisition_date']=pd.to_datetime(alldata.created_dt,
                                                  infer_datetime_format=True)

In [190]:
 alldata['acquisition_date']=alldata['acquisition_date'].apply(lambda x: x.date())

In [193]:
alldata.groupby('acquisition_date').sum().to_csv('acquisition_facts.csv',
                                                columns=['acquisitions'])

In [192]:
alldata.groupby('acquisition_date').sum()

,is_unsub,acquisitions
acquisition_date,,
1970-01-01,8.0,13
1970-01-02,13.0,14
1970-01-03,7.0,11
1970-01-04,17.0,18
1970-01-05,15.0,17
1970-01-06,15.0,17
1970-01-07,15.0,16
1970-01-08,17.0,17
1970-01-09,10.0,11
